# U.S. Medical Insurance Costs

# Goals
Observations:
1. Things to note: There are categorical and numerical data
2. No data is missing
3. There are 7 columns, variables

Possible goals / questions:
1. What is the average age of insured people?
2. Is there a relationship between age and cost of insurance?
3. Is insurance more expensive if you're a female?
4. Where do most insured people live?
5. Is insurance more expensive depending on your region?
6. Look at the differences in cost between smokers vs non-smokers
7. Average age of people with at least one child


# Data

In [20]:
import csv
from pprint import pprint

In [26]:
# Read in the csv file and store the contents into separate
# lists for each column
# Example: BMI column -> bmi [ ... ]

# Initialise all the lists
ages, sexes, bmis, children, smokers, regions, charges = [], [], [], [], [], [], []

with open('insurance.csv') as insurance_file:
    insurance_csv = csv.DictReader(insurance_file)
    
    for item in insurance_csv:
        ages.append(item['age'])
        sexes.append(item['sex'])
        bmis.append(item['bmi'])
        children.append(item['children'])
        smokers.append(item['smoker'])
        regions.append(item['region'])
        charges.append(item['charges'])

In [28]:
print(ages[:5])
print(sexes[:5])
print(bmis[:5])
print(children[:5])
print(smokers[:5])
print(regions[:5])
print(charges[:5])

['19', '18', '28', '33', '32']
['female', 'male', 'male', 'male', 'male']
['27.9', '33.77', '33', '22.705', '28.88']
['0', '1', '3', '0', '0']
['yes', 'no', 'no', 'no', 'no']
['southwest', 'southeast', 'southeast', 'northwest', 'northwest']
['16884.924', '1725.5523', '4449.462', '21984.47061', '3866.8552']


In [31]:
# TODO Create function to convert number strings into ints or floats


# Analysis